# Clean Global Goals data (official descriptions) and save result in DB

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from google_drive_downloader import GoogleDriveDownloader as gdd
from pathlib import Path
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
import re
import string
import pprint

In [4]:
import stanza
from spacy_stanza import StanzaLanguage

- Global variables

In [5]:
svStanzaDownloaded = False
snlpInitialized = False
nlpStanza = {}

pp = pprint.PrettyPrinter(indent=2, width=200)

categories = []

- Needed to be able to run on __gpu (cuda)__

In [6]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
table_name = 'Goals'
db_path = 'db/goals.sqlite'
db_path = 'sqlite:///' + db_path

In [8]:
engine = create_engine(db_path, echo=True)
data_df = pd.read_sql_table(table_name, engine)
data_df.set_index('Id')

2020-12-15 21:30:28,205 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-15 21:30:28,206 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 21:30:28,208 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-15 21:30:28,208 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 21:30:28,210 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-12-15 21:30:28,211 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 21:30:28,213 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='view' ORDER BY name
2020-12-15 21:30:28,213 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 21:30:28,215 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("Goals")
2020-12-15 21:30:28,216 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 21:30:28,219 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL 

,Name,Description
Id,,
1,Ingen fattigdom,Ingen fattigdom\n\n\nFattigdom omfattar fler d...
2,Ingen hunger,Ingen hunger\n\n\nHunger är den främsta dödsor...
3,God hälsa och välbefinnande,God hälsa och välbefinnande\n\n\nGod hälsa är ...
4,God utbildning för alla,God utbildning för alla\n\n\nUtbildning är en ...
5,Jämställdhet,Jämställdhet\n\n\nJämställdhet mellan kvinnor ...
6,Rent vatten och sanitet för alla,Rent vatten och sanitet för alla\n\n\nVatten ä...
7,Hållbar energi för alla,Hållbar energi för alla\n\n\nTillgång till hål...
8,Anständiga arbetsvillkor och ekonomisk tillväxt,Anständiga arbetsvillkor och ekonomisk tillväx...
9,"Hållbar industri, innovationer och infrastruktur","Hållbar industri, innovationer och infrastrukt..."


In [9]:
data_df = data_df.rename(columns={"Description": "abstract", "Name": "title"})

## Data cleaning

1. Clean text round 1
    - transform to lower case
    - replace punctuation with spaces
2. Clean text round 2
    - remove new lines (replace with a single space)
    - remove some punctuation and non-sensical text that was missed in round 1
3. Lemmatize text


In [10]:
def initStanzaPipeline(lang):
    downloadStanza(lang)
    global snlpInitialized
    global nlpStanza
    if not snlpInitialized:
        snlp = stanza.Pipeline(lang=lang)
        nlpStanza['snlp'] = StanzaLanguage(snlp)
        snlpInitialized = True

In [11]:
def removeNewlines(text):
    text = re.sub(r'\\r\\n', ' ', text)
    text = re.sub(r'\\n', ' ', text)
    return ' '.join(text.split())

In [12]:
def clean_text_round1(text):
    """First round of cleaning.

    Make text lowercase, remove text in square brackets,
    remove punctuation and remove words containing numbers.
    """
    text = removeNewlines(text)
    text = text.lower()
    text = re.sub(r'|[.*?]|', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

In [13]:
data_df

,Id,title,abstract
0,1,Ingen fattigdom,Ingen fattigdom\n\n\nFattigdom omfattar fler d...
1,2,Ingen hunger,Ingen hunger\n\n\nHunger är den främsta dödsor...
2,3,God hälsa och välbefinnande,God hälsa och välbefinnande\n\n\nGod hälsa är ...
3,4,God utbildning för alla,God utbildning för alla\n\n\nUtbildning är en ...
4,5,Jämställdhet,Jämställdhet\n\n\nJämställdhet mellan kvinnor ...
5,6,Rent vatten och sanitet för alla,Rent vatten och sanitet för alla\n\n\nVatten ä...
6,7,Hållbar energi för alla,Hållbar energi för alla\n\n\nTillgång till hål...
7,8,Anständiga arbetsvillkor och ekonomisk tillväxt,Anständiga arbetsvillkor och ekonomisk tillväx...
8,9,"Hållbar industri, innovationer och infrastruktur","Hållbar industri, innovationer och infrastrukt..."
9,10,Minskad ojämlikhet,Minskad ojämlikhet\n\n\nGrunden för ett hållba...


In [14]:
data_clean = pd.DataFrame(data_df.abstract.apply(clean_text_round1))

In [15]:
print(data_clean.abstract)

0     ingen fattigdom fattigdom omfattar fler dimens...
1     ingen hunger hunger är den främsta dödsorsaken...
2     god hälsa och välbefinnande god hälsa är en gr...
3     god utbildning för alla utbildning är en grund...
4     jämställdhet jämställdhet mellan kvinnor och m...
5     rent vatten och sanitet för alla vatten är en ...
6     hållbar energi för alla tillgång till hållbar ...
7     anständiga arbetsvillkor och ekonomisk tillväx...
8     hållbar industri innovationer och infrastruktu...
9     minskad ojämlikhet grunden för ett hållbart sa...
10    hållbara städer och samhällen över hälften av ...
11    hållbar konsumtion och produktion vår planet h...
12    bekämpa klimatförändringarna klimatförändringa...
13    hav och marina resurser världens hav – deras t...
14    ekosystem och biologisk mångfald hållbara ekos...
15    fredliga och inkluderande samhällen fredliga s...
16    genomförande och globalt partnerskap världen ä...
Name: abstract, dtype: object


In [16]:
def clean_text_round2(text):
    """Second round of cleaning.

    Get rid of some additional punctuation and non-sensical text
    that was missed the first time around.
    """
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[‘’“”…–|-]', '', text)
    text = re.sub(r'\s\s+', ' ', text)
    text = re.sub(r'syfte och mål', '', text)
    text = re.sub(r'mål|syftet|syfte', '', text)
    text = re.sub(r'konferensen |konferens |projektet |projekt ', '', text)
    return text

In [17]:
data_clean = pd.DataFrame(data_clean.abstract.apply(clean_text_round2))

In [18]:
print(data_clean.abstract)

0     ingen fattigdom fattigdom omfattar fler dimens...
1     ingen hunger hunger är den främsta dödsorsaken...
2     god hälsa och välbefinnande god hälsa är en gr...
3     god utbildning för alla utbildning är en grund...
4     jämställdhet jämställdhet mellan kvinnor och m...
5     rent vatten och sanitet för alla vatten är en ...
6     hållbar energi för alla tillgång till hållbar ...
7     anständiga arbetsvillkor och ekonomisk tillväx...
8     hållbar industri innovationer och infrastruktu...
9     minskad ojämlikhet grunden för ett hållbart sa...
10    hållbara städer och samhällen över hälften av ...
11    hållbar konsumtion och produktion vår planet h...
12    bekämpa klimatförändringarna klimatförändringa...
13    hav och marina resurser världens hav deras tem...
14    ekosystem och biologisk mångfald hållbara ekos...
15    fredliga och inkluderande samhällen fredliga s...
16    genomförande och globalt partnerskap världen ä...
Name: abstract, dtype: object


In [19]:
def downloadStanza(lang):
    # download sv stanza (to avoid downloading it for every doc)
    global svStanzaDownloaded
    if not svStanzaDownloaded:
        stanza.download(lang)
        svStanzaDownloaded = True

In [20]:
def lemmatizeText(text):
    initStanzaPipeline('sv')
    doc = nlpStanza['snlp'](text)
    lemmatized_text = ''
    for token in doc:
        if token.lemma_ == 'all':
            token.lemma_ = 'alla'
        lemmatized_text += token.lemma_ + ' '
    text = lemmatized_text
    return text

In [21]:
data_clean = pd.DataFrame(data_clean.abstract.apply(lemmatizeText))

2020-12-15 21:31:01 INFO: Downloading default packages for language: sv (Swedish)...
2020-12-15 21:31:02 INFO: File exists: /home/nesko/stanza_resources/sv/default.zip.
2020-12-15 21:31:06 INFO: Finished downloading models and saved to /home/nesko/stanza_resources.
2020-12-15 21:31:06 INFO: Loading these models for language: sv (Swedish):
| Processor | Package   |
-------------------------
| tokenize  | talbanken |
| pos       | talbanken |
| lemma     | talbanken |
| depparse  | talbanken |

2020-12-15 21:31:06 INFO: Use device: gpu
2020-12-15 21:31:06 INFO: Loading: tokenize
2020-12-15 21:31:12 INFO: Loading: pos
2020-12-15 21:31:13 INFO: Loading: lemma
2020-12-15 21:31:13 INFO: Loading: depparse
2020-12-15 21:31:15 INFO: Done loading processors!


In [30]:
data_df = data_df.drop(columns=['abstract'])

In [34]:
data_df = data_df.drop(columns=['title'])

In [38]:
data_df = data_df.rename(columns={"Id": "index"})

In [44]:
data_df

,goals
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [45]:
data_merged = pd.concat([data_df, data_clean], axis=1)
data_merged

,goals,abstract
0,1,ingen fattigdom fattigdom omfatta fler dimensi...
1,2,ingen hung hung vara en främst dödsorsak i vär...
2,3,god hälsa och välbefinnande god hälsa vara en ...
3,4,god utbildning för alla utbildning vara en gru...
4,5,jämställdhet jämställdhet mellan kvinna och ma...
5,6,ren vatten och sanitet för alla vatten vara en...
6,7,hållbar energi för alla tillgång till hållbar ...
7,8,anständig arbetsvillkor och ekonomisk tillväxt...
8,9,hållbar industri innovation och infrastruktur ...
9,10,minska ojämlikhet grund för en hållbar samhäll...


In [46]:
table_name_cleaned = 'clean_goals'
db_path = 'db/categorised_swecris.sqlite'
db_path = 'sqlite:///' + db_path
engine = create_engine(db_path, echo=True)

In [47]:
data_merged.to_sql(table_name_cleaned, engine, if_exists='fail')

2020-12-15 22:00:42,645 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-15 22:00:42,646 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 22:00:42,647 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-15 22:00:42,648 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 22:00:42,649 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("clean_goals")
2020-12-15 22:00:42,650 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 22:00:42,651 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("clean_goals")
2020-12-15 22:00:42,652 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 22:00:42,654 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE clean_goals (
	"index" BIGINT, 
	goals BIGINT, 
	abstract TEXT
)


2020-12-15 22:00:42,655 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 22:00:42,665 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-15 22:00:42,666 INFO sqlalchemy.engine.base.